In [8]:
import cv2
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as functional
import torch.optim as optim
from PIL import Image

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt

In [9]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super(ConvolutionalNetwork, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=9, kernel_size=5)
        self.conv1_bn = nn.BatchNorm2d(9)
        self.conv2 = nn.Conv2d(in_channels=9, out_channels=18, kernel_size=5)
        self.conv2_bn = nn.BatchNorm2d(18)

        self.fc1 = nn.Linear(in_features=18*9*9, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=64)
        self.out = nn.Linear(in_features=64, out_features=7)
        
    def forward(self, t):
        t = self.conv1(t)
        t = self.conv1_bn(t)
        t = functional.relu(t)
        t = functional.max_pool2d(t, kernel_size=2, stride=2)
        
        t = self.conv2(t)
        t = self.conv2_bn(t)
        t = functional.relu(t)
        t = functional.max_pool2d(t, kernel_size=2,  stride=2)

        t = functional.relu(self.fc1(t.reshape(-1, 18*9*9)))
        t = functional.relu(self.fc2(t))
        t = self.out(t)
        
        return t

In [12]:
def take_photo():
    cam = cv2.VideoCapture(0)
    s, img = cam.read()
    if s:
        cv2.namedWindow("cam-test", cv2.WINDOW_NORMAL)
        cv2.imshow("cam-test", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    crop = img[200:400, 250:450]
    cv2.imwrite("cam-photo.jpg", crop)


def image_to_tensor(fname):
    pil_img = Image.open(fname).convert('L') # convert to greyscale
    pil_img.thumbnail((48,48))
    t = transforms.ToTensor()(pil_img)
    t = torch.stack([t for i in range(32)])
    return t

def guess_emotion(t):
    network = ConvolutionalNetwork()
    network.load_state_dict(torch.load("C:/Users/Nikhil Deenamsetty/MachineLearning/NNModels/EmotionCNNV1.pth", map_location='cpu'))
    labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
    with torch.no_grad():
        out = network(t)
        _, predicted = torch.max(out.data, 1)
    
    return labels[predicted[0]]

In [14]:
take_photo()
t = image_to_tensor("cam-photo.jpg")
guess_emotion(t)

'Happy'